# MAP ALEGBRA

In [46]:
import numpy as np
import pandas as pd
from scipy.ndimage.filters import generic_filter
from numba import jit

In [21]:
c = np.random.randint(0, 10, 9)
indata_sm = np.reshape(c, (3,3))
indata_sm

array([[1, 8, 4],
       [0, 2, 9],
       [3, 8, 9]])

In [30]:
# write a null in the middle
indata = np.where(indata_sm==2, np.nan, indata_sm) 
indata

array([[ 1.,  8.,  4.],
       [ 0., nan,  9.],
       [ 3.,  8.,  9.]])

Replace ```nan```(null) values with the mean from the array

In [29]:
# The middle value is calculated based on 
# its surrounding neighbors in the input dataset 
np.where(np.isnan(indata),
         np.nanmean(indata),indata)

array([[1.  , 8.  , 4.  ],
       [0.  , 5.25, 9.  ],
       [3.  , 8.  , 9.  ]])

Now we will use the 3 x 3 matrix as a map or "filter footprint" and apply it to a "raster" matrix to eliminate ```nan``` values. This process is called smoothing.

#### A random-generated raster dataset:

In [37]:
a = np.random.randint(0, 10, 64)
indata01 = np.reshape(a, (8,8))
indata_w_null = np.where(indata01==0, np.nan, indata01) # replace all zero values with nan
indata_w_null

array([[nan,  7.,  1.,  2.,  1.,  2.,  6.,  7.],
       [ 6.,  6.,  6.,  5.,  3.,  7.,  8.,  1.],
       [ 2.,  9.,  7.,  6.,  4.,  9.,  2.,  9.],
       [nan, nan,  3.,  9.,  1.,  5.,  6.,  7.],
       [ 7.,  2.,  8.,  6.,  2.,  1.,  9.,  6.],
       [ 2.,  1.,  8.,  7.,  2.,  5., nan,  1.],
       [ 1., nan,  3.,  7.,  8.,  7.,  9.,  9.],
       [ 4.,  4.,  1.,  4.,  3.,  9.,  6.,  8.]])

### SciPy Generic Filter
We must pass the following parameters:
* footprint - a 3 x 3 array
* function - what our convolution should do
* mode - this has to do with what to do at the borders

See docs for more info: https://docs.scipy.org/doc/scipy-0.16.1/reference/generated/scipy.ndimage.filters.generic_filter.html

In [50]:
@jit
def conv_mapping(x):
    """
    If the forth value of the filter array (the center of the window) 
    is nan, replace it with the mean of the surrounding values.
    """
    if np.isnan(x[4]) and not np.isnan(np.delete(x, 4)).all():
        return np.nanmean(np.delete(x, 4))
    else:
        return x[4]

In [39]:
# our filter footprint
fp=np.ones((3, 3))

In [43]:
generic_filter(indata_w_null,conv_mapping,footprint=fp, mode='constant', cval=np.NaN)

array([[6.33333333, 7.        , 1.        , 2.        , 1.        ,
        2.        , 6.        , 7.        ],
       [6.        , 6.        , 6.        , 5.        , 3.        ,
        7.        , 8.        , 1.        ],
       [2.        , 9.        , 7.        , 6.        , 4.        ,
        9.        , 2.        , 9.        ],
       [5.        , 5.42857143, 3.        , 9.        , 1.        ,
        5.        , 6.        , 7.        ],
       [7.        , 2.        , 8.        , 6.        , 2.        ,
        1.        , 9.        , 6.        ],
       [2.        , 1.        , 8.        , 7.        , 2.        ,
        5.        , 5.875     , 1.        ],
       [1.        , 3.        , 3.        , 7.        , 8.        ,
        7.        , 9.        , 9.        ],
       [4.        , 4.        , 1.        , 4.        , 3.        ,
        9.        , 6.        , 8.        ]])

In [45]:
# compare the "smooth" above to original below
indata_w_null

array([[nan,  7.,  1.,  2.,  1.,  2.,  6.,  7.],
       [ 6.,  6.,  6.,  5.,  3.,  7.,  8.,  1.],
       [ 2.,  9.,  7.,  6.,  4.,  9.,  2.,  9.],
       [nan, nan,  3.,  9.,  1.,  5.,  6.,  7.],
       [ 7.,  2.,  8.,  6.,  2.,  1.,  9.,  6.],
       [ 2.,  1.,  8.,  7.,  2.,  5., nan,  1.],
       [ 1., nan,  3.,  7.,  8.,  7.,  9.,  9.],
       [ 4.,  4.,  1.,  4.,  3.,  9.,  6.,  8.]])

## Now let's do it at scale with Numba

In [2]:
df = pd.read_csv("data/cell_data.csv")
df1 = df.drop(columns=['celda_id', 'Unnamed: 0'])
cells = df1.set_index(['x', 'y']).sort_index()

In [3]:
cells.shape

(98558, 3)

In [4]:
cells[cells.isnull()==False].shape

(98558, 3)

In [5]:
raster = cells.to_xarray()

In [14]:
@jit
def conv_mapping(x):
    if np.isnan(x[4]) and not np.isnan(np.delete(x, 4)).all():
        return np.nanmean(np.delete(x, 4))
    else:
        return x[4]

In [51]:
@jit
def smoothing(dframe):
    for col in dframe.columns[dframe.columns != 'celda_id'].tolist():
        raster[col].values = generic_filter(
            input=raster[col].values,
            function=conv_mapping,
            footprint=np.ones((3, 3)), mode='constant', cval=np.NaN)
    return raster

In [16]:
solution = smoothing(cells)

In [17]:
solution.coords

Coordinates:
  * x        (x) float64 3.331e+05 3.331e+05 3.331e+05 ... 3.431e+05 3.431e+05
  * y        (y) float64 6.185e+06 6.185e+06 6.185e+06 ... 6.19e+06 6.19e+06

In [18]:
solution.isel

<bound method Dataset.isel of <xarray.Dataset>
Dimensions:     (x: 973, y: 1050)
Coordinates:
  * x           (x) float64 3.331e+05 3.331e+05 ... 3.431e+05 3.431e+05
  * y           (y) float64 6.185e+06 6.185e+06 6.185e+06 ... 6.19e+06 6.19e+06
Data variables:
    VELOCIDAD   (x, y) float64 nan nan nan nan nan nan ... nan nan nan nan nan
    DOSIS OBJE  (x, y) float64 nan nan nan nan nan nan ... nan nan nan nan nan
    DOSIS SEMI  (x, y) float64 nan nan nan nan nan nan ... nan nan nan nan nan>

In [ ]:
raster.VELOCIDAD.plot()

In [ ]:
solution.VELOCIDAD.plot()

In [75]:
cells.describe()

,VELOCIDAD,DOSIS OBJE,DOSIS SEMI
count,98558.000000,98558.000000,98558.000000
mean,6.993043,34.319030,33.666604
std,1.390845,5.456466,6.640882
min,0.000000,0.000000,0.000000
25%,6.500000,32.000000,30.300000
50%,6.833333,36.000000,35.466667
75%,7.075000,38.666667,38.500000
max,241.700000,42.000000,81.950000


In [69]:
solution.describe()

,VELOCIDAD,DOSIS OBJE,DOSIS SEMI
count,541182.000000,541182.000000,541182.000000
mean,6.940606,34.231799,33.340936
std,2.048461,5.950976,7.314550
min,0.000000,0.000000,0.000000
25%,6.450000,32.000000,30.138889
50%,6.800000,36.000000,35.450000
75%,7.037500,38.833333,38.500000
max,241.700000,42.000000,81.950000


In [99]:
vel = solution.VELOCIDAD
vel

x              y           
333053.607111  6.184531e+06   NaN
               6.184531e+06   NaN
               6.184531e+06   NaN
               6.184546e+06   NaN
               6.184546e+06   NaN
                               ..
343133.607111  6.189616e+06   NaN
               6.189616e+06   NaN
               6.189631e+06   NaN
               6.189631e+06   NaN
               6.189646e+06   NaN
Name: VELOCIDAD, Length: 1021650, dtype: float64

AttributeError: 'Series' object has no attribute 'coords'

These two forms of viewing our footprint are equivalent.

In [18]:
# form one
raster_simple = np.arange(9)
raster_simple

array([0, 1, 2, 3, 4, 5, 6, 7, 8])

In [19]:
# form two
np.reshape(raster_simple, (3,3))

array([[0, 1, 2],
       [3, 4, 5],
       [6, 7, 8]])